In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
df = pd.read_csv('Student_performance_data _.csv')

print(df)

      StudentID  Age  Gender  Ethnicity  ParentalEducation  StudyTimeWeekly  \
0          1001   17       1          0                  2        19.833723   
1          1002   18       0          0                  1        15.408756   
2          1003   15       0          2                  3         4.210570   
3          1004   17       1          0                  3        10.028829   
4          1005   17       1          0                  2         4.672495   
...         ...  ...     ...        ...                ...              ...   
2387       3388   18       1          0                  3        10.680555   
2388       3389   17       0          0                  1         7.583217   
2389       3390   16       1          0                  2         6.805500   
2390       3391   16       1          1                  0        12.416653   
2391       3392   16       1          0                  2        17.819907   

      Absences  Tutoring  ParentalSupport  Extracur

## Pulizia

In [6]:
# Rimuovo eventuali duplicati
df = df.drop_duplicates()

# Valori nulli
print(df.isna().sum()) #non ce ne sono

#variabili categoriche mettiamo minuscolo e tolgo eventuali spazi vuoti all'inizio e alla fine della riga
cat_cols = df.select_dtypes(include=['object']).columns
for c in cat_cols:
    df[c] = df[c].str.strip().str.lower()

# separo numeriche e categoriche dopo la pulizia
num_cols=df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

StudentID            0
Age                  0
Gender               0
Ethnicity            0
ParentalEducation    0
StudyTimeWeekly      0
Absences             0
Tutoring             0
ParentalSupport      0
Extracurricular      0
Sports               0
Music                0
Volunteering         0
GPA                  0
GradeClass           0
dtype: int64


## Normalizzazione

In [8]:
# Normalizzo variabili numeriche
scaler = StandardScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df[num_cols]), columns=num_cols, index=df.index)

## encoding

In [10]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc = encoder.fit_transform(df[cat_cols])
enc_cols = encoder.get_feature_names_out(cat_cols)
df_enc = pd.DataFrame(enc,columns=enc_cols, index=df.index)

In [11]:
# unisco normalizzazione ed encoding
df_tot = pd.concat([df_norm, df_enc], axis=1)
print('Shape originale:', df.shape)
print('Shape trasformata:', df_tot.shape)
print(df_tot.head())

Shape originale: (2392, 15)
Shape trasformata: (2392, 15)
   StudentID       Age    Gender  Ethnicity  ParentalEducation  \
0  -1.731327  0.472919  0.978492  -0.853391           0.253711   
1  -1.729879  1.362944 -1.021981  -0.853391          -0.746087   
2  -1.728430 -1.307132 -1.021981   1.091641           1.253509   
3  -1.726982  0.472919  0.978492  -0.853391           1.253509   
4  -1.725534  0.472919  0.978492  -0.853391           0.253711   

   StudyTimeWeekly  Absences  Tutoring  ParentalSupport  Extracurricular  \
0         1.780336 -0.890822  1.522371        -0.108744        -0.788476   
1         0.997376 -1.717694 -0.656870        -0.999551        -0.788476   
2        -0.984045  1.353542 -0.656870        -0.108744        -0.788476   
3         0.045445 -0.063951 -0.656870         0.782063         1.268269   
4        -0.902311  0.290422  1.522371         0.782063        -0.788476   

     Sports     Music  Volunteering       GPA  GradeClass  
0 -0.660132  2.019544     -0